In [23]:
import numpy as np
import matplotlib.pyplot as plt
from CAMB import camb

In [24]:
wmap = np.loadtxt('Intensity.txt')

In [25]:
#Copied the spectrum function written by Prof. Sievers
def get_spectrum(pars,lmax=2000,fixed_Tau=None):
    if fixed_Tau is None:
        H0=pars[0]
        ombh2=pars[1]
        omch2=pars[2]
        tau=pars[3]
        As=pars[4]
        ns=pars[5]
    else: 
        H0=pars[0]
        ombh2=pars[1]
        omch2=pars[2]
        tau=fixed_Tau
        As=pars[3]
        ns=pars[4]
    pars=camb.CAMBparams()
    pars.set_cosmology(H0=H0,ombh2=ombh2,omch2=omch2,mnu=0.06,omk=0,tau=tau)
    pars.InitPower.set_params(As=As,ns=ns,r=0)
    pars.set_for_lmax(lmax,lens_potential_accuracy=0)
    results=camb.get_results(pars)
    powers=results.get_cmb_power_spectra(pars,CMB_unit='muK')
    cmb=powers['total']
    tt=cmb[:,0]    #you could return the full power spectrum here if you wanted to do say EE
    return tt

In [38]:
class MCMC: 
    def take_step_cov(self,covmat):
        mychol = np.linalg.cholesky(covmat)
        return np.dot(mychol,np.random.randn(covmat.shape[0]))
    
    
    def run_chain(self,x,y,p,errP,nstep,pcov):
        j = 0
        chains = np.zeros([nstep,len(p)])
        chisqvec = np.zeros(nstep)
        chisq = np.sum((x-y(p)[2:len(x)+2])**2/errP**2)
        scale_fac = 0.5
        for i in range(nstep):
            new_p = p + self.take_step_cov(pcov)*scale_fac
            if new_p[3]>0:
                new_m = y(new_p)[2:len(x)+2]
                new_chisq = np.sum((x-new_m)**2/errP**2)

                cond = new_chisq - chisq
                prob = np.exp(-0.5*cond)
                accept = np.random.rand(1)<prob
                if accept:
                    j+=1
                    p = new_p 
                    m = new_m
                    chisq = new_chisq
            chains[i,:] = p
            chisqvec[i] = chisq
        return chains,chisqvec,p

In [39]:
#Defined from the previous question 
pcov = np.asarray([[8.27963540e+00,1.38059605e-03,-1.50966395e-02,2.31683686e-01,8.17793756e-10,4.25805390e-02],[1.38059605e-03,4.47664949e-07,-1.64546691e-06,5.76110907e-05,2.19546969e-13,1.10298580e-05]
 ,[-1.50966395e-02,-1.64546691e-06,3.43333369e-05,-3.77088120e-04,-1.24887877e-12,-6.36488736e-05],[2.31683686e-01,5.76110907e-05,-3.77088120e-04,2.07377984e-02,7.84029301e-11,1.80406820e-03]
 ,[8.17793756e-10,2.19546969e-13,-1.24887877e-12,7.84029301e-11,2.97761352e-19,6.77396391e-12],[4.25805390e-02,1.10298580e-05,-6.36488736e-05,1.80406820e-03,6.77396391e-12,3.40209983e-04]])

In [ ]:
pars=np.asarray([65,0.02,0.1,0.05,2e-9,0.96]) 
mcmc = MCMC()
c,chi,p=mcmc.run_chain(wmap[:,1],get_spectrum,pars,wmap[:,2],5000,pcov)

In [87]:
file_name = 'MCMC_results_chain.txt'
file_name_1 = 'MCMC_results_chi.txt'
file_name_2 = 'MCMC_results_params.txt'
np.savetxt(file_name,c)
np.savetxt(file_name_1,chi)
np.savetxt(file_name_2,p)